In [1]:
import pandas as pd
# import sqlite3
import sqlite3
pd.options.mode.chained_assignment = None

from dateutil import parser
from datetime import datetime, timedelta
from itertools import compress

import numpy as np

from dateutil import parser

#Loading the streak data set
streak=pd.read_csv('tags_2908.csv')

streak['date'] = streak.apply(lambda x: parser.parse(x['created_at']).date(), axis = 1)

def datedeltas(df):
  day = pd.Timedelta('1d')
  in_block = (df.date.diff(-1) == -day) | (df.date.diff() == day)
  return in_block

org = list(set(streak['org_id']))[1]

itemsmnt = streak.pivot_table(columns="org_id",aggfunc="sum").transpose().reset_index()

itemsmnt = itemsmnt[['item_amount', 'org_id','item_quantity']]
itemsmnt.columns = ['total_amount_tagged','org_id','product_qnty_sold']

dfs = []
for org in set(streak['org_id']):
  try:
    df = streak.loc[streak.org_id == org]
    df.drop_duplicates(['date'],inplace=True)
    df['inblock'] = datedeltas(df)
    day = pd.Timedelta('1d')
    breaks = df['date'].diff() != day
    df['groups'] = breaks.cumsum()
    df = df[['date','groups','org_id']]
    df2 = df.copy()
    df2 = pd.DataFrame(df2.pivot(values = 'date', columns= ['groups']))
    df2 = df2.transpose()
    df2 = df2.set_index(df2.index.rename('group')).fillna(0).reset_index()
    strk = [[a!=0 for a in df2.loc[df2.group==x].values.flatten().tolist()[1:]]\
            for x in list(df2.group)]
    data = [list(compress(["a"]*len(a), a)) for a in strk]
    ctrk = any([a == (datetime.now() - timedelta(days=1)).date() for a in list(df['date'])])
    if ctrk:
      loc = [a for a,b in enumerate(df['date']) if b == (datetime.now() - timedelta(days=1)).date()][0]
      grp = list(df['groups'])[loc]
      ss = len(data[grp-1])
    dfs.append({"org_id": org,
        "total_days_active": len(strk[0]),
        "number_of streaks": len(strk),
        "longest_streak": max([len(a) for a in data]),
        "shortest_streak": min([len(a) for a in data]),
        "mean_streak": np.mean([len(a) for a in data]),
        "current_streak": ctrk,
        "current_streak_duration": ss if ctrk else 0,
        "last_streak": list(df['date'])[-1].strftime("%d-%m-%Y")})
  except:
    continue

dfs = pd.DataFrame(dfs)

dfsmain = dfs.merge(itemsmnt,on='org_id')

org_dets = pd.read_csv("org_details_2608.csv")

dfsmain = dfsmain.merge(org_dets,on='org_id')

dfsmain.to_csv('streak_data_3008.csv')

dfsmain.head(22)



import requests
import pandas as pd

data = requests.get("https://api.nobuk.africa/auth/metrics")

reqdata = data.json()

df_login = pd.DataFrame([x.split(":") for x in [a.split(" ")[0] for a in reqdata['data']]],columns=["org_id","date"])

df_login

df_login2=df_login.merge(org_dets,on="org_id")

df_login2.head()

df = df_login[['org_id', 'date']]
df['args'] = df.apply(lambda x: f"{x['org_id']}-{x['date']}", axis=1)

df.drop_duplicates('args',inplace=True)

df.pivot_table(values="org_id",columns="date",aggfunc="count").transpose()

df = streak.loc[streak.org_id == org]
df.drop_duplicates(['date'],inplace=True)
df['inblock'] = datedeltas(df)
day = pd.Timedelta('1d')

breaks = df['date'].diff() != day
df['groups'] = breaks.cumsum()
df = df[['date','groups','org_id']]
df2 = df.copy()
df2 = pd.DataFrame(df2.pivot(values = 'date', columns= ['groups']))
df2 = df2.transpose()
df2 = df2.set_index(df2.index.rename('group')).fillna(0).reset_index()
strk = [[a!=0 for a in df2.loc[df2.group==x].values.flatten().tolist()[1:]] \
        for x in list(df2.group)]
data = [list(compress(["a"]*len(a), a)) for a in strk]

df4 = df.merge(org_dets,on="org_id")

df4

dfsmain.head()

import os
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication
import os
import urllib
import  requests
import json

def sendMail():
  smtp_ssl_host = 'smtp.gmail.com'  # smtp.mail.yahoo.com
  smtp_ssl_port = 465
  username = 'edwinokwaro3@gmail.com'
  password = 'dzdsesnzderrvubc'   # this is the appa password for gmail
  sender = 'edwinokwaro3@gmail.com'
  targets = ['edwinokwaro3@gmail.com','piuskanuti@gmail.com']

  msg = MIMEMultipart()
  msg['Subject'] = 'Streak Data'
  msg['From'] = sender
  msg['To'] = ', '.join(targets)

  txt = MIMEText('please find the attached streak data.')
  msg.attach(txt)

  filepath = 'streak_data_3008.csv'
  with open(filepath, 'rb') as f:
    msg.attach(MIMEApplication(f.read(), Name="streak_data_3008.csv"))
  
  server = smtplib.SMTP_SSL(smtp_ssl_host, smtp_ssl_port)
  server.login(username, password)
  server.sendmail(sender, targets, msg.as_string())
sendMail()
from apscheduler.schedulers.blocking import BlockingScheduler
from nobuk import sendMail
sched = BlockingScheduler()

@sched.scheduled_job('interval', seconds=10)
def timed_job():
    print('This job is run every one minute.')
    sendMail()
sched.start()

FileNotFoundError: ignored